In [1]:
label_dict={"with_mask": 0,"without_mask": 1}

In [2]:
categories=["with_mask","without_mask"]

In [3]:
labels=[0,1]

In [4]:
data_path="D:\\Verzeo\\Facemask\\dataset"

In [5]:
import os
import cv2

In [6]:
data=[]
target=[]

In [7]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try: 
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            pass

In [8]:
import numpy as np
import tensorflow as tf

In [9]:
data=np.array(data)

In [10]:
data=data/255

In [11]:
data=np.reshape(data,(data.shape[0],100,100,1))

In [12]:
data.shape


(1376, 100, 100, 1)

In [13]:
data.shape[1:]

(100, 100, 1)

In [14]:
target=np.array(target)

In [15]:
target.shape

(1376,)

In [16]:
from keras.utils import np_utils

In [17]:
new_target=np_utils.to_categorical(target)

In [18]:
new_target.shape

(1376, 2)

In [19]:
from keras.models import Sequential

In [20]:
from keras.layers import Dense, Activation, Flatten, Dropout

In [21]:
from keras.layers import Conv2D,MaxPooling2D

In [22]:
model= Sequential()

In [23]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:],activation="relu")) #it helps us to detect edges
model.add(MaxPooling2D(pool_size=(2,2))) #reduce the number of pixel by keeping max pixel value
model.add(Conv2D(100,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [24]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [27]:
from keras.callbacks import ModelCheckpoint

In [28]:
checkpoint=ModelCheckpoint("model-{epoch:03d}.model",save_best_only=True,mode="auto")
model.fit(train_data,train_target,epochs=40,validation_split=0.2,callbacks=[checkpoint])

Epoch 1/40
31/31 [==============================] - ETA: 0s - loss: 0.7037 - accuracy: 0.5343WARNING:tensorflow:From C:\Users\AMD\anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model\assets
31/31 [==============================] - 31s 986ms/step - loss: 0.7037 - accuracy: 0.5343 - val_loss: 0.6757 - val_accuracy: 0.5927
Epoch 2/40
31/31 [==============================] - 31s 999ms/step - loss: 0.5187 - accuracy: 0.7444 - val_loss: 0.3592 - val_accuracy: 0.8669
Epoch 3/40
31/31 [==============================] - 31s 996ms/step - loss: 0.3491 - accuracy: 0.8414

In [29]:
t_image="D:\\test" 

In [30]:
test=[]

In [31]:
timg_names=os.listdir(t_image)
for timg_name in timg_names:
    timg_path=os.path.join(t_image,timg_name)
    timg=cv2.imread(timg_path)
    try:
        tgray=cv2.cvtColor(timg,cv2.COLOR_BGR2GRAY)
        tresized=cv2.resize(tgray,(100,100))
        test.append(tresized)
      
    except Exception as e:
        pass

In [32]:
test=np.array(test)

In [33]:
test=test/255

In [34]:
test=np.reshape(test,(test.shape[0],100,100,1))

In [35]:
probability_model=tf.keras.Sequential([model,tf.keras.layers.Softmax()])

In [36]:
prediction=probability_model.predict(test)

In [53]:
c=0
for i in prediction:
    lab=np.argmax(i)
    c=c+1
    print(c,"image is",categories[lab])
   


1 image is without_mask
2 image is with_mask


In [54]:
np.argmax(prediction[1])

0

In [55]:
lab=np.argmax(prediction[1])

In [56]:
#xml file
face_cascader=cv2.CascadeClassifier("D:\\Verzeo\\14th Lecture-20200908T074323Z-001\\14th Lecture\\haarcascade_frontalface_default.xml")

In [57]:
#specific image
img=cv2.imread("D:\\test\\mask.jpg")
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces=face_cascader.detectMultiScale(gray)
for(x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(img,(x,y-40),(x+w,y),(0,0,255),2)
    cv2.putText(img,categories[lab],(x+2,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),2)
cv2.imshow("img",img)
cv2.waitKey(2)


-1

In [44]:
#videoCapture
cap=cv2.VideoCapture("D:\\test\\face.mp4") #0 to capture webcame
while True:
    _,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascader.detectMultiScale(gray)
    for(x,y,w,h) in faces:
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        
        normimage=resized/255
        reshapeimage=np.reshape(normimage,(-1,100,100,1))
        modelop=model.predict(reshapeimage)
        
        label=np.argmax(modelop,axis=1)[1]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],1)
        
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
       
        
    cv2.imshow("checking...",img)
    key=cv2.waitKey(2)
    
    if(key==27):
        break
cap.release()

KeyboardInterrupt: 

In [ ]:
#webcam
color_dict={0:(0,255,0),1:(0,0,255)}
source=cv2.VideoCapture(0)
while(True):

    ret,img=source.read()
    
    faces=face_cascader.detectMultiScale(img,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        
        
        
        normimage=resized/255
        reshapeimage=np.reshape(normimage,(-1,100,100,1))
        modelop=model.predict(reshapeimage)
        
        label=np.argmax(modelop,axis=1)[1]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],1)
        
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
       
        
    cv2.imshow("checking...",img)
    key=cv2.waitKey(2)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()